In [1]:
%load_ext autoreload
%autoreload 2

from data_reading import read_images, load_reference_poses, read_imu, sync_data
from visualization import visualize_trajectory
from visual_odometry import compute_relative_pose, compute_matches, add_vo_factors
from imu_preintegration import add_imu_factors

imgs = read_images()
imu = read_imu()
gt_poses = load_reference_poses()
imgs_list = []
for i, img in enumerate(imgs):
    if i>1000:
        break
    imgs_list.append(img)
synced_data = sync_data(imu, imgs_list, gt_poses)

/home/forge/Desktop/cs179-final/LightGlue/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [2]:
points0, points1, descriptors, E = compute_matches(synced_data[0].img_left_start, synced_data[0].img_right_start)
t, r = compute_relative_pose(E, points0, points1)
print(synced_data[0].gt_pose_start)
estimated_second_pose_t = synced_data[0].gt_pose_start[1].apply(t) + synced_data[0].gt_pose_start[0]
estimated_second_pose_r = synced_data[0].gt_pose_start[1] * r

pos_diff = estimated_second_pose_t - synced_data[0].gt_pose_end[0]
rot_diff = estimated_second_pose_r.as_euler('XYZ', degrees=True) - synced_data[0].gt_pose_end[1].as_euler('XYZ', degrees=True)
print(f"Position difference: {pos_diff}")
print(f"Rotation difference: {rot_diff}")


1036 inliers found out of 1136 matches
(array([ 0.1762211 , -0.05578694,  0.00300311]), <scipy.spatial.transform._rotation.Rotation object at 0x74880869b960>)
Position difference: [-0.01359674 -0.05951615  0.99813473]
Rotation difference: [ 4.48969005e-05  8.36231169e-05 -1.57638936e-04]
